In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
from datetime import date
from tqdm.notebook import tqdm

In [2]:
def get_keywords_from_url(film_url):

    url = film_url+'keywords'
    req = requests.get(url)
    status_code = req.status_code
    if status_code == 200:
        soup = BeautifulSoup(req.text, 'html.parser')
        keywords_list = [link.get('data-real-keyword') for link in soup.find_all('a') if link.get('data-real-keyword')]
        keywords_string = ','.join(keywords_list)
    else:
        keywords_list = ['']
        keywords_string = ''

    print(f'URL:{url}, status: {status_code}, keywords: {keywords_list}')

    return url, status_code, keywords_string

In [8]:
film_url = 'https://www.kinopoisk.ru/level/1/film/426275/'
url, status_code, keywords_string = get_keywords_from_url(film_url)

URL:https://www.kinopoisk.ru/level/1/film/426275/keywords, status: 200, keywords: []


In [11]:
url = film_url+'keywords'
req = requests.get(url)
status_code = req.status_code
soup = BeautifulSoup(req.text, 'html.parser')
keywords_list = [link.get('data-real-keyword') for link in soup.find_all('a') if link.get('data-real-keyword')]
keywords_string = ','.join(keywords_list)

In [15]:
data = [link.get('data-real-keyword') for link in soup.find_all('a')]

In [17]:
data = soup.find_all('a')

In [18]:
data

[<a class="Link Link_view_default LogoLink" href="https://kinopoisk.ru" title="Kinopoisk"></a>,
 <a class="Link Link_view_default" href="https://yandex.ru/support/smart-captcha?unique_key=c147efa-f13ad5fe-175dbee4-e6a6ed87">ÐÐ¾ÑÐµÐ¼Ñ ÑÑÐ¾ Ð¼Ð¾Ð³Ð»Ð¾ Ð¿ÑÐ¾Ð¸Ð·Ð¾Ð¹ÑÐ¸?</a>,
 <a class="Link Link_view_default" href="https://yandex.ru/support/common/browsers-settings/browsers-java-js-settings.html">ÐÐ°Ðº Ð²ÐºÐ»ÑÑÐ¸ÑÑ JavaScript?</a>,
 <a aria-describedby="service-link-description" class="Link Link_view_captcha" href="https://cloud.yandex.ru/services/smartcaptcha" target="_blank">Yandex SmartCaptcha</a>,
 <a aria-describedby="connect-link-description" class="Link Link_view_captcha" href="https://yandex.ru/legal/smartcaptcha_notice/" target="_blank">ÐÐ±ÑÐ°Ð±Ð¾ÑÐºÐ° Ð´Ð°Ð½Ð½ÑÑ
 </a>,
 <a class="Link Link_view_default" href="https://yandex.ru/support/smart-captcha?unique_key=c147efa-f13ad5fe-175dbee4-e6a6ed87#help">ÑÐ¾ÑÐ¼Ð¾Ð¹ Ð¾Ð±ÑÐ°ÑÐ½Ð¾Ð¹ ÑÐ²ÑÐ·Ð¸</a>]

In [3]:
def get_keywords_from_df(df):

    df.insert(4,'keywords','')
    df.insert(5,'keywords_status','')

    current_date = str.replace(str(date.today()), '-', '_')
    current_time = time.strftime('%H_%M_%S', time.localtime())
    temporal_txt_path = 'fil_word_scraper_temp_file_' + \
        current_date + '_' + current_time + '.csv'

    for film_url in tqdm(df.film_url):
        
        url, status_code, keywords = get_keywords_from_url(film_url)
        df.loc[df.film_url == film_url, 'keywords'] = keywords
        df.loc[df.film_url == film_url, 'keywords_status'] = status_code

        with open(temporal_txt_path, 'a') as temp_file:
            temp_file.write(
                ';'.join([str(film_url), str(keywords), str(status_code)])+'\n')

        time.sleep(2) 

    df.to_csv('fil_word_scraper.csv', sep=';')    

    return df

In [4]:
df_fil_scraper = pd.read_csv('fil_scraper.csv', sep=';', index_col=0)
df_to_word_scraper = df_fil_scraper.iloc[11:20,:]
df_to_word_scraper

,film_full_name,year,counry,film_url
11,Господа Головлевы (видео),(2006),Россия,https://www.kinopoisk.ru/level/1/film/426275/
12,Брат 2,(2000),Россия,https://www.kinopoisk.ru/level/1/film/41520/
13,Щенок,(2009),Россия,https://www.kinopoisk.ru/level/1/film/478759/
14,Евгений Гришковец: Дредноуты (видео),(2006),Россия,https://www.kinopoisk.ru/level/1/film/425999/
15,Евгений Гришковец: Планета (видео),(2005),Россия,https://www.kinopoisk.ru/level/1/film/425998/
16,Дурак,(2014),Россия,https://www.kinopoisk.ru/level/1/film/808639/
17,Радости и печали маленького лорда,(2003),Россия,https://www.kinopoisk.ru/level/1/film/40699/
18,Солдатик,(2018),Россия,https://www.kinopoisk.ru/level/1/film/1254051/
19,В августе 44-го,(2001),Россия,https://www.kinopoisk.ru/level/1/film/41381/


In [5]:
df = get_keywords_from_df(df_to_word_scraper)

  0%|          | 0/9 [00:00<?, ?it/s]

URL:https://www.kinopoisk.ru/level/1/film/426275/keywords, status: 200, keywords: []
URL:https://www.kinopoisk.ru/level/1/film/41520/keywords, status: 200, keywords: []
URL:https://www.kinopoisk.ru/level/1/film/478759/keywords, status: 200, keywords: []
URL:https://www.kinopoisk.ru/level/1/film/425999/keywords, status: 200, keywords: []
URL:https://www.kinopoisk.ru/level/1/film/425998/keywords, status: 200, keywords: []
URL:https://www.kinopoisk.ru/level/1/film/808639/keywords, status: 200, keywords: []
URL:https://www.kinopoisk.ru/level/1/film/40699/keywords, status: 200, keywords: []
URL:https://www.kinopoisk.ru/level/1/film/1254051/keywords, status: 200, keywords: []
URL:https://www.kinopoisk.ru/level/1/film/41381/keywords, status: 200, keywords: []


In [6]:
df

,film_full_name,year,counry,film_url,keywords,keywords_status
11,Господа Головлевы (видео),(2006),Россия,https://www.kinopoisk.ru/level/1/film/426275/,,200
12,Брат 2,(2000),Россия,https://www.kinopoisk.ru/level/1/film/41520/,,200
13,Щенок,(2009),Россия,https://www.kinopoisk.ru/level/1/film/478759/,,200
14,Евгений Гришковец: Дредноуты (видео),(2006),Россия,https://www.kinopoisk.ru/level/1/film/425999/,,200
15,Евгений Гришковец: Планета (видео),(2005),Россия,https://www.kinopoisk.ru/level/1/film/425998/,,200
16,Дурак,(2014),Россия,https://www.kinopoisk.ru/level/1/film/808639/,,200
17,Радости и печали маленького лорда,(2003),Россия,https://www.kinopoisk.ru/level/1/film/40699/,,200
18,Солдатик,(2018),Россия,https://www.kinopoisk.ru/level/1/film/1254051/,,200
19,В августе 44-го,(2001),Россия,https://www.kinopoisk.ru/level/1/film/41381/,,200


In [7]:
df

,film_full_name,year,counry,film_url,keywords,keywords_status
11,Господа Головлевы (видео),(2006),Россия,https://www.kinopoisk.ru/level/1/film/426275/,,200
12,Брат 2,(2000),Россия,https://www.kinopoisk.ru/level/1/film/41520/,,200
13,Щенок,(2009),Россия,https://www.kinopoisk.ru/level/1/film/478759/,,200
14,Евгений Гришковец: Дредноуты (видео),(2006),Россия,https://www.kinopoisk.ru/level/1/film/425999/,,200
15,Евгений Гришковец: Планета (видео),(2005),Россия,https://www.kinopoisk.ru/level/1/film/425998/,,200
16,Дурак,(2014),Россия,https://www.kinopoisk.ru/level/1/film/808639/,,200
17,Радости и печали маленького лорда,(2003),Россия,https://www.kinopoisk.ru/level/1/film/40699/,,200
18,Солдатик,(2018),Россия,https://www.kinopoisk.ru/level/1/film/1254051/,,200
19,В августе 44-го,(2001),Россия,https://www.kinopoisk.ru/level/1/film/41381/,,200
